# Satellite elevation-azimuth diagrams and PDOP
### Guillaume Witz, Science IT Support, Bern University

## Import packages

In [1]:
import os, re, glob
from datetime import date, timedelta

import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
from bz2 import BZ2File as bzopen

import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual

import aiub

## User parameters

In [2]:
#file location
address = 'http://ftp.aiub.unibe.ch/BSWUSER52/STA/2019/'
#address_sat = 'http://ftp.aiub.unibe.ch/CODE/'
address_sat = 'http://ftp.aiub.unibe.ch/CODE_MGEX/CODE/'
#station coordinates file
coord_file = 'COD19001.CRD.Z'
#station to observe
station_name = '14001M004'

## Elevation-azimuth
#### Data import and computation

In [5]:
#import station data
stations = aiub.import_stations(address,coord_file)

In [6]:
#select station and calculate ellipsoidal coordinates
curr_stat = stations[stations.statname == station_name].iloc[0]
rad_stat_coord = aiub.cartesian_to_ellipsoidal(curr_stat.x_pos, curr_stat.y_pos,curr_stat.z_pos)
curr_stat['rad_stat_coord'] = rad_stat_coord

In [6]:
date_min, date_max = aiub.date_picker()

In [7]:
#create a list of dates between start and end date 
date_list = [date_min.value + timedelta(days=x) for x in range((date_max.value-date_min.value).days+1)]
#calculate gps weeks times (year, week, day)
gps_weeks = [aiub.date_to_gpsweeks(x) for x in date_list]

#load satellite data
temp_pd = pd.concat([aiub.import_RM_file(address_sat, g) for g in gps_weeks]).reset_index()

#reanme satellite types
sat_dict = {'G':'GPS','R':'GLONASS','E':'Galileo','C':'BeiDou','J':'QZSS'}
temp_pd['satellite'] = temp_pd.satellite.apply(lambda x : sat_dict[x])

In [8]:
#caluclate elevation and azimuth for all satellites for given station
temp_pd = aiub.el_al_single_station_fast(temp_pd, curr_stat)
temp_pd['curr_stat'] = curr_stat.statname

### Plotting

In [9]:
aiub.interactive_rad(temp_pd, stations)

Output()

## PDOP

### User input

In [10]:
date_min, date_max = aiub.date_picker()

### Data import and calculation

In [20]:
#create a list of dates between start and end date 
date_list = [date_min.value + timedelta(days=x) for x in range((date_max.value-date_min.value).days+1)]
#calculate gps weeks times (year, week, day)
gps_weeks = [aiub.date_to_gpsweeks(x) for x in date_list]

#load satellite data and calculate el, az
temp_pd = pd.concat([aiub.import_RM_file(address_sat, g) for g in gps_weeks]).reset_index()
temp_pd = aiub.el_al_single_station_fast(temp_pd, curr_stat)


In [21]:
#remove elevation < 5°
temp_pd = temp_pd[temp_pd.e >2*np.pi*5/360]


In [22]:
#calculate norm of satellite-station vector Xgeo
temp_pd['Xgeo_norm'] = temp_pd.apply(lambda row: np.linalg.norm([row['Xgeo'],row['Ygeo'],row['Zgeo']]),axis = 1)


In [23]:
time_grouped = temp_pd.set_index('datetime').groupby(pd.Grouper(freq='30min'))

In [24]:
#calculate pdop for each group
pdop = dict((key, []) for key in temp_pd.satellite.unique())

for name, df0 in time_grouped:
    sat_grouped = df0.groupby('satellite')
    for name2, df in sat_grouped:
        A_mat = np.stack([-df['Xgeo']/df['Xgeo_norm'],-df['Ygeo']/df['Xgeo_norm'],
              -df['Zgeo']/df['Xgeo_norm'],np.ones(len(df))],axis = 1)

        try:
            inv_mat = np.linalg.inv(np.matmul(A_mat.T,A_mat))
        except:
            False

        pdop[name2].append(np.sqrt(inv_mat[0,0]+inv_mat[1,1]+inv_mat[2,2]))

/Users/gw18g940/miniconda3/envs/astronomy/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


In [83]:
# calculate power spectrum and set the right (?) scale 
powerspec = {}
for x in pdop.keys():
    powerspec[x] = scipy.signal.periodogram(pdop[x],fs = 48, scaling='spectrum')
    

### Plotting

In [84]:
#define interactive plotting function 
def plot_pdop(sat_type, pdop, powerspec):
    fig, ax = plt.subplots(2,1)
    for s in sat_type:
        ax[0].plot(np.arange(len(pdop[s]))/(24*2),pdop[s], label = s)
        
        ax[1].plot(powerspec[s][0],powerspec[s][1],'-o', label = s)
    ax[0].legend()
    
    #ax[1].set_xlim
    plt.xscale('log')
    ax[1].set_xlim((0,10))
    plt.show()

In [85]:
ipw.interactive(plot_pdop, sat_type = ipw.SelectMultiple(options =temp_pd.satellite.unique(),index=[0]),
                pdop = ipw.fixed(pdop), powerspec = ipw.fixed(powerspec))

interactive(children=(SelectMultiple(description='sat_type', index=(0,), options=('G', 'R', 'E', 'C'), value=(…